In [1]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [2]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -q git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 29.7 MB/s 
     |████████████████████████████████| 18.7 MB 360 kB/s 
     |████████████████████████████████| 143 kB 25.3 MB/s 
     |████████████████████████████████| 4.9 MB 54.1 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 55.9 MB 181 kB/s 
     |████████████████████████████████| 5.3 MB 54.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.8 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 648 kB/s 
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 112 kB 70.7 MB/s 
     |████████████████████████████████| 270 kB 76.2 MB/s 
     |████████

### Install xformers from precompiled wheel.

In [3]:
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

     |████████████████████████████████| 102.9 MB 85 kB/s 


## Login to HF and run

In [4]:
#@markdown Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [5]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown Path for images of the concept for training.
INSTANCE_DIR = "/content/data/sks" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown A general name for class like dog for dog images.
CLASS_NAME = "woman" #@param {type:"string"}
CLASS_DIR = f"/content/data/{CLASS_NAME}"

#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "stable_diffusion_weights/sks" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

#@markdown sks is a rare identifier, feel free to replace it.

Mounted at /content/drive
[*] Weights will be saved at /content/drive/MyDrive/stable_diffusion_weights/sks


In [ ]:
#@markdown Upload your (5-6) images by running this cell.

#@markdown OR

#@markdown You can use the file manager on the left panel to upload (drag and drop) to INSTANCE_DIR (it uploads faster)

import os
from google.colab import files
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
    dst_path = os.path.join(INSTANCE_DIR, filename)
    shutil.move(filename, dst_path)

### Start training

remove `--use_8bit_adam` flag in the cell below if you got more than equal to 18 GB VRAM.

`--mixed_precision="fp16"` uses a little more VRAM for some reason but is 1.5-2x faster on GPUs with tensor cores, change it to `--mixed_precision="no"` if you face out of memory issues.

In [6]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME --use_auth_token \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="photo of sks {CLASS_NAME}" \
  --class_prompt="photo of a {CLASS_NAME}" \
  --seed=1337 \
  --resolution=512 \
  --center_crop \
  --train_batch_size=1 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=12 \
  --sample_batch_size=4 \
  --max_train_steps=800


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Downloading: 100% 543/543 [00:00<00:00, 545kB/s]
Fetching 16 files:   0% 0/16 [00:00<?, ?it/s]
Downloading: 100% 342/342 [00:00<00:00, 298kB/s]
Fetching 16 files:   6% 1/16 [00:02<00:34,  2.28s/it]
Downloading: 100% 4.56k/4.56k [00:00<00:00, 3.82MB/s]
Fetching 16 files:  19% 3/16 [00:04<00:18,  1.45s/it]
Downloading:   0% 0.00/1.22G [00:00<?, ?B/s]
Downloading:   1% 7.88M/1.22G [00:00<00:15, 78.8MB/s]
Downloading:   1% 15.8M/1.22G [00:00<00:15, 77.9MB/s]
Downloading:   2% 23.6M/1.22G [00:00<00:15, 77.0MB/s]
Downloading:   3% 31.3M/1.22G [00:00<00:15, 75.1MB/s]
Downloading:   3%

## Inference

In [7]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

model_path = OUTPUT_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
g_cuda = None

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [8]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 523662 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [ ]:
#@title Run for generating images.

prompt = "sks woman pastel pink big bouffant hair bow satin pussy bridal blouse huge ruffle knickers wide 50s double petticoat puffy diaper nappy pinup stepford wife pink pinafore dress high detail sissy 4k 8k uhd intricate " #@param {type:"string"}
num_samples = 10 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe([prompt] * num_samples, height=512, width=512, num_inference_steps=200, guidance_scale=9, generator=g_cuda).images

for img in images:
    display(img)

  0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
#@markdown Run Gradio UI for generating images.
import gradio as gr

def inference(prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
            [prompt]*int(num_samples), height=int(height), width=int(width),
            num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
            generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of sks guy, digital painting")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://16116.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


  0%|          | 0/51 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f592a118a10>,
 'http://127.0.0.1:7860/',
 'https://16116.gradio.app')